In [1]:
import os
import json
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import trimesh

In [ ]:
class CameraDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): path to dataset root
            transform (callable, optional): optional transforms applied to RGB
        """
        self.root_dir = root_dir
        self.rgb_dir = os.path.join(root_dir, "rgb")
        self.depth_dir = os.path.join(root_dir, "depth")

        # Load camera matrices
        with open(os.path.join(root_dir, "camera.json"), 'r') as f:
            cam_data = json.load(f)
            self.cam_matrices = cam_data["camera_c2w"]

        self.frame_ids = sorted([entry["frame"] for entry in self.cam_matrices])
        self.transform = transform

    def __len__(self):
        return len(self.frame_ids)

    def __getitem__(self, idx):
        frame_id = self.frame_ids[idx]
        rgb_path = os.path.join(self.rgb_dir, f"{frame_id:04d}.png")
        depth_path = os.path.join(self.depth_dir, f"{frame_id:04d}.exr")
        
        # Load RGB image
        rgb = cv2.imread(rgb_path, cv2.IMREAD_UNCHANGED)[:, :, :3]  # drop alpha if present
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

        # Load depth image (.exr is 32-bit float)
        depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)  # (H, W)
        if depth.ndim == 3:  # handle (H, W, 3) case
            depth = depth[:, :, 0]

        # Get camera-to-world matrix
        c2w = np.array(self.cam_matrices[idx]["matrix"], dtype=np.float32)

        sample = {
            "rgb": torch.from_numpy(rgb).permute(2, 0, 1),  # (3, H, W)
            "depth": torch.from_numpy(depth),               # (H, W)
            "c2w": torch.from_numpy(c2w),                   # (4, 4)
            "frame_id": frame_id
        }

        if self.transform:
            sample["rgb"] = self.transform(sample["rgb"])

        return sample

In [ ]:
# dataset = CameraDataset("/Users/gemmechu/Documents/meta/data/cube")
# # loader = DataLoader(dataset, batch_size=1, shuffle=False)

# for batch in loader:
#     print("RGB:", batch["rgb"].shape)
#     print("Depth:", batch["depth"].shape)
#     print("Camera Pose:\n", batch["c2w"])
#     break

In [9]:
data_dir = "/Users/gemmechu/Documents/meta/data/cube/points"
for fname in sorted(os.listdir(data_dir)):
    fpath = os.path.join(data_dir,fname)
    pts = trimesh.load(fpath).vertices
    

(250000, 3)


In [6]:
from scipy.spatial import cKDTree

data_dir = "/Users/gemmechu/Documents/meta/data/cube/points_color"
thresh, merged = 0.01, None
results = []
for f in sorted(os.listdir(data_dir)):
    pts = trimesh.load(os.path.join(data_dir, f)).vertices
    if merged is None:
        merged = pts
    else:
        tree = cKDTree(merged)
        mask = np.isinf(tree.query(pts, distance_upper_bound=thresh)[0])
        merged = np.vstack((merged, pts[mask]))
        results.append(merged)
i = 0
for f in sorted(os.listdir(data_dir)):
    _ = trimesh.PointCloud(results[i]).export(f"output/all_color/{f}")
    i +=1 
    

In [14]:
import os, trimesh, numpy as np
from scipy.spatial import cKDTree

data_dir = "/Users/gemmechu/Documents/meta/data/cube/points_color"
out_dir = "output/all_color"
os.makedirs(out_dir, exist_ok=True)

merged_pts, merged_colors = None, None
for f in sorted(os.listdir(data_dir)):
    pc = trimesh.load(os.path.join(data_dir, f))
    pts, colors = pc.vertices, getattr(pc, 'colors', None)

    if merged_pts is None:
        merged_pts, merged_colors = pts, colors
    else:
        mask = np.isinf(cKDTree(merged_pts).query(pts, distance_upper_bound=0.01)[0])
        merged_pts = np.vstack((merged_pts, pts[mask]))
        if merged_colors is not None and colors is not None:
            merged_colors = np.vstack((merged_colors, colors[mask]))

    out_pc = trimesh.PointCloud(merged_pts, colors=merged_colors) if merged_colors is not None else trimesh.PointCloud(merged_pts)
    _ = out_pc.export(os.path.join(out_dir, f))


In [9]:
i = 0
for f in sorted(os.listdir(data_dir)):
    _ = trimesh.PointCloud(results[i]).export(f"output/all_color/{f}")
    i +=1 
    

IndexError: list index out of range

In [14]:
def voxel_downsample(points, voxel_size):
    coords = (points / voxel_size).astype(np.int32)
    _, unique_idx = np.unique(coords, axis=0, return_index=True)
    return points[unique_idx]

data_dir = "/Users/gemmechu/Documents/meta/data/cube/points"
voxel_size = 0.01
all_points = []

for f in sorted(os.listdir(data_dir)):
    pts = trimesh.load(os.path.join(data_dir, f)).vertices
    all_points.append(pts)

merged = voxel_downsample(np.vstack(all_points), voxel_size)
trimesh.PointCloud(merged).export("output/merged2.ply")